In [339]:
from tqdm.notebook import tqdm
from collections import defaultdict
from queue import PriorityQueue
import bitstring
import json
import matplotlib.pyplot as plt
import math
import random

In [493]:
class HuffmanNode:

   def __init__(self, name, value, bit=None, parent=None):
      self.name = name
      self.value = value
      self.parent = parent
      self.bit = bit

   def __lt__(self, obj):
      return self.value < obj.value

   def __str__(self):
      parent_val = "None" if self.parent is None else self.parent.name
      return f"Name: {self.name}, Value: {self.value}, Bit: {self.bit}, Parent: {parent_val}"

class Huffman:

   def __init__(self):
      pass

   def encode_translation_table(self, translation_table):
      encode = ""
      for key, value in translation_table.items():
         encode += "{0:08b}".format(len(key))
         encode += key
         encode += "{0:08b}".format(value)
      return bitstring.BitArray(bin=encode).tobytes()

   def decode_translation_table(self, translation_table):
      binary = ""
      for byte in translation_table:
         binary += "{0:08b}".format(byte)
      reverse_translation_table = {}
      i = 0
      while i < len(binary):
         if i+8 > len(binary):
            break
         key_size = int(binary[i:i+8], 2)
         if i+16+key_size > len(binary):
            break
         key = binary[i+8:i+8+key_size]
         value = int(binary[i+8+key_size:i+16+key_size], 2)
         reverse_translation_table[key] = value
         i += 16+key_size
      return reverse_translation_table

   def encode(self, message):
      assert type(message)==bytes, "Message should be Byte-String"
      pq = PriorityQueue()
      counts = defaultdict(int)
      base_nodes = {}
      for byte in message:
         counts[byte] += 1
      for name, value in counts.items():
         huffman_node = HuffmanNode(name, value)
         pq.put(huffman_node)
         base_nodes[name] = huffman_node
      while pq.qsize() > 1:
         left_node = pq.get()
         right_node = pq.get()
         left_node.bit, right_node.bit = "0", "1"
         internal_node = HuffmanNode(f"( {left_node.name}, {right_node.name} )",
                                    left_node.value+right_node.value)
         left_node.parent, right_node.parent = internal_node, internal_node
         pq.put(internal_node)
      translation, reverse_translation = dict(), dict()
      for name, node in base_nodes.items():
         translate = ""
         curr_node = node
         while curr_node.bit is not None:
            translate = curr_node.bit + translate
            curr_node = curr_node.parent
         translation[name] = translate
         reverse_translation[translate] = name
      print("Encoding Message...")
      encoded = "".join(translation[byte] for byte in tqdm(message))
      extra = 0
      if len(encoded)%8!=0:
         extra = 8 - len(encoded)%8
      encoded += "0"*extra
      reverse_translation_encode = self.encode_translation_table(reverse_translation)
      preamble = "{0:03b}".format(extra) + "{0:013b}".format(len(reverse_translation_encode))
      preamble = bitstring.BitArray(bin=preamble).tobytes()
      encoded = bitstring.BitArray(bin=encoded).tobytes()
      encoded = preamble + reverse_translation_encode + encoded
      return encoded

   def decode(self, encoded):
      # try:
      binary = ""
      for byte in encoded:
         binary += "{0:08b}".format(byte)
      extra = int(binary[:3], 2)
      reverse_len = int(binary[3:16], 2)
      reverse_translation = self.decode_translation_table(encoded[2:2+reverse_len])
      message = binary[(2+reverse_len)*8:len(binary)-extra]
      i = 0
      decoded = ""
      max_len = max(len(key) for key in reverse_translation)
      print("Decoding the message...")
      pb = tqdm(total=len(message))
      while i < len(message):
         collect = ""
         while i < len(message) and collect not in reverse_translation:
            collect += message[i]
            assert len(collect)<=max_len
            pb.update(1)
            i += 1
         if collect in reverse_translation:
            byte = reverse_translation[collect]
            byte = "{0:08b}".format(byte)
            decoded += byte
      assert len(decoded)%8 == 0
      decoded = bitstring.BitArray(bin=decoded).tobytes()
      return decoded
      # except Exception as e:
      #    print(f"Encoding Scheme Does not Match!! {e}")
      #    return b""

In [498]:
filename = "./ProfileSm.tin"
with open(filename, "rb") as f:
   raw = f.read()

huffman = Huffman()
encoded = huffman.encode(raw)
with open(filename+".hf", "wb") as f:
   f.write(encoded)

assert raw == huffman.decode(encoded), "Original and Decoded message Do Not Match!"
compression_ratio = 100*(1-len(encoded)/len(raw))
compression_ratio

Encoding Message...


  0%|          | 0/125805 [00:00<?, ?it/s]

Decoding the message...


  0%|          | 0/581523 [00:00<?, ?it/s]

41.995946107070466